## Запуск моделей в облаке

Вчера мы использовали облачные LLM-модели. При этом выбор моделей был ограничен, и мы не могли использовать какие-то экспериментальные модели, до-обученные энтузиастами, или нами. В облаке также есть вычислительные ресурсы - машины с GPU - которые позволяют запускать любые не слишком тяжеловесные модели.

Большой каталог моделей на разные случаи жизни есть на портале [HuggingFace](http://huggingface.com). Их работа обеспечивается библиотеками `transformers` (для текстовых моделей) и `diffusers` (для картинок).

**Для запуска в Datasphere важно выполнить следующую ячейку**. Для работы в Google Colab этого лучше не делать.

In [ ]:
%pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118
%pip install transformers accelerate

Следующую ячейку необходимо выполнить и в Google Colab:

In [5]:
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.5 MB/s eta 0:00:00


С помощью библиотеки transformers мы можем легко загружать модели с портала HuggingFace - достаточно взять оттуда фрагмент кода и его использовать (возможно, с минимальными изменениями). При этом сами модели автоматически будут скачаны и использованы.

Возьмём для примера [модель Llama 3, до-обученную на русском датасете Saiga](https://huggingface.co/IlyaGusev/saiga_llama3_8b).

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_path = "IlyaGusev/saiga_llama3_8b"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

messages = [
    {"role": "system", "content": "Ты помощник, задача которого - вежливо отвечать на вопросы. Используй бодный тон."},
    {"role": "user", "content": "Расскажи анекдот про Python"}
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Конечно! Вот один из известных анекдотов про язык программирования Python:

Почему Python не хочет играть в шахматы?
Потому что он всегда предпочитает "if" делать ставки!

Надеюсь, вам понравился!


Чтобы дальше общаться с этой моделью и сохранять историю переписки, опишем небольшой класс `Agent`:

In [3]:
class Agent:
  def __init__(self,instruction):
    self.messages = [
       {"role": "system", "content": instruction}
    ]
    self.generation_args = {
      "max_new_tokens": 500,
      "return_full_text": False,
      "temperature": 0.3,
      "do_sample": True,
    }

  def __call__(self,x):
    self.messages.append({ "role" : "user", "content" : x })
    res = pipe(self.messages,**self.generation_args)
    res = res[0]['generated_text']
    self.messages.append({ "role" : "assistant", "content" : res })
    return res

teacher = Agent("Ты учитель математики по имени Мисс Радиус. Отвечай на вопросы ученика")

teacher("Здравствуйте! Я хочу узнать, что такое число Пи.")

'Здравствуйте! Число Пи — это математическая константа, которая представляет собой отношение длины окружности к диаметру круга. Это число примерно равно 3.14159 (но оно не рациональное и имеет бесконечные десятичные цифры после запятой). Важность числа Пи заключается в том, что оно используется во многих математических формулах, особенно в геометрии и арифметике. Например, формула для площади круга: A = πr^2, где r — радиус круга. Надеюсь, теперь вы лучше понимаете значение этого важного числа!'

In [4]:
teacher("А если округлить его до целого?")

'Если округлить число Пи до ближайшего целого, мы получаем примерно 3. Но это очень грубое приближение, так как реальное значение числа Пи гораздо больше. Если бы мы округлили его до двух знаков после запятой, то получили бы примерно 3.14. Но важно помнить, что точный расчет чисел Пи требует сложных алгоритмов и может занять много времени.'

Реализуем диалог двух агентов между собой - учителя и ученика:

In [5]:
student = Agent("Ты ученик 7-го класса, который хочет узнать больше про математику. Ты разговариваешь с учителем.  Используй короткие разговорные фразы.")
teacher = Agent("Ты учитель математики по имени Мисс Радиус. Отвечай на вопросы ученика 7-го класса. Используй короткие разговорные фразы.")

msg = "Здравствуйте! Я хочу узнать, что такое число Пи."
for _ in range(5):
  print(f"Ученик: {msg}")
  msg = teacher(msg)
  print(f"Учитель: {msg}")
  msg = student(msg)

Ученик: Здравствуйте! Я хочу узнать, что такое число Пи.
Учитель: Привет! Число Пи — это особое число, которое связано с окружностью и дугами кругов. Оно примерно равно 3.14159 (но точное значение очень большое). Это число используется в геометрии, математике и даже в технике.
Ученик: Да, точно! Я знал, что число Пи важное для математики и науки. А как оно применяется в реальной жизни? Например, когда мы строим здания или автомобили?
Учитель: Конечно! Число Пи помогает в архитектуре, инженерии и производстве. Например, при строительстве колонн, арочных мостов или шасси автомобилей учитываются дуги и радиусы, которые напрямую связаны с числом Пи. Без него сложно было бы создавать такие конструкции.
Ученик: Ого, это действительно важно! А какие еще примеры использования числа Пи есть? Может быть, в компьютерах или интернет-трафике?
Учитель: Да, конечно! Число Пи также применяется в компьютерных сетях и алгоритмах. Например, в криптографии, где зашифрованные данные передаются через Интерн

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Учитель: Число Пи — это не просто цифры; это символ знаний, открытий и прогресса человечества. Его изучение привело к множеству научных и технических прорывов. Каждый год в мире проводится праздник Pi Day (21 марта), чтобы увековечить его значимость.
Ученик: Праздники всегда веселее! Действительно, число Пи заслуживает такого внимания. Спасибо за объяснение, теперь я лучше понимаю, почему это число так важно!
Учитель: Рад был помочь! Если у тебя возникнут ещё вопросы, не стесняйся задавать их. Удачи тебе во всех твоих начинаниях!


Все сообщения доступны в поле `messages`:

In [6]:
teacher.messages

[{'role': 'system',
  'content': 'Ты учитель математики по имени Мисс Радиус. Отвечай на вопросы ученика 7-го класса. Используй короткие разговорные фразы.'},
 {'role': 'user',
  'content': 'Здравствуйте! Я хочу узнать, что такое число Пи.'},
 {'role': 'assistant',
  'content': 'Привет! Число Пи — это особое число, которое связано с окружностью и дугами кругов. Оно примерно равно 3.14159 (но точное значение очень большое). Это число используется в геометрии, математике и даже в технике.'},
 {'role': 'user',
  'content': 'Да, точно! Я знал, что число Пи важное для математики и науки. А как оно применяется в реальной жизни? Например, когда мы строим здания или автомобили?'},
 {'role': 'assistant',
  'content': 'Конечно! Число Пи помогает в архитектуре, инженерии и производстве. Например, при строительстве колонн, арочных мостов или шасси автомобилей учитываются дуги и радиусы, которые напрямую связаны с числом Пи. Без него сложно было бы создавать такие конструкции.'},
 {'role': 'user',


## Модели синтеза речи

На портале HuggingFace можно найти разные интересные модели, например, модель для синтеза речи от Suno, создателей инструмента генерации музыки. Попробуем синтезировать простую фразу:

In [7]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-speech", "suno/bark-small")

speech = synthesiser("Привет! Моя собака круче тебя!", forward_params={"do_sample": True})


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Чтобы послушать результат, нужно использовать небольшую Python-магию:

In [8]:
from IPython.display import Audio

Audio(speech['audio'], rate=speech['sampling_rate'])

Всё это можем обернуть в одну функцию:

In [18]:
def synthesize(x):
  speech = synthesiser(x, forward_params={"do_sample": True})
  return Audio(speech['audio'],rate=speech['sampling_rate'])

synthesize(teacher.messages[1]['content'])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Другой способ синтеза речи - это использовать какую-нибудь библиотеку, например, Silero Voice:

In [19]:
%pip install silero-tts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 8.9 MB/s eta 0:00:00


Посмотрим, какие модели доступны:

In [20]:
from silero_tts.silero_tts import SileroTTS

models = SileroTTS.get_available_models()
print("Available models:", models)

2025-09-30 23:30:18.143 | WARNING  | silero_tts.silero_tts:get_available_models:309 - Models config file not found: /usr/local/lib/python3.12/dist-packages/silero_tts/latest_silero_models.yml. Downloading...
2025-09-30 23:30:18.442 | SUCCESS  | silero_tts.silero_tts:download_models_config_static:361 - Models config file downloaded: /usr/local/lib/python3.12/dist-packages/silero_tts/latest_silero_models.yml


Available models: {'ru': ['v4_ru', 'v3_1_ru', 'ru_v3', 'aidar_v2', 'aidar_8khz', 'aidar_16khz', 'baya_v2', 'baya_8khz', 'baya_16khz', 'irina_v2', 'irina_8khz', 'irina_16khz', 'kseniya_v2', 'kseniya_8khz', 'kseniya_16khz', 'natasha_v2', 'natasha_8khz', 'natasha_16khz', 'ruslan_v2', 'ruslan_8khz', 'ruslan_16khz'], 'en': ['v3_en', 'v3_en_indic', 'lj_v2', 'lj_8khz', 'lj_16khz'], 'de': ['v3_de', 'thorsten_v2', 'thorsten_8khz', 'thorsten_16khz'], 'es': ['v3_es', 'tux_v2', 'tux_8khz', 'tux_16khz'], 'fr': ['v3_fr', 'gilles_v2', 'gilles_8khz', 'gilles_16khz'], 'ba': ['aigul_v2'], 'xal': ['v3_xal', 'erdni_v2'], 'tt': ['v3_tt', 'dilyara_v2'], 'uz': ['v4_uz', 'v3_uz', 'dilnavoz_v2'], 'ua': ['v4_ua', 'v3_ua', 'mykyta_v2'], 'indic': ['v4_indic', 'v3_indic'], 'cyrillic': ['v4_cyrillic'], 'multi': ['multi_v2']}


И какие спикеры:

In [25]:
tts = SileroTTS(model_id='v4_ru', language='ru', sample_rate=48000, device='cuda')
tts.get_available_speakers()


2025-09-30 23:33:05.080 | SUCCESS  | silero_tts.silero_tts:load_models_config:48 - Models config loaded from: /usr/local/lib/python3.12/dist-packages/silero_tts/latest_silero_models.yml
2025-09-30 23:33:05.084 | INFO     | silero_tts.silero_tts:init_model:148 - Initializing model
2025-09-30 23:33:05.084 | INFO     | silero_tts.silero_tts:init_model:156 - Using 1 GPU(s)...
2025-09-30 23:33:05.085 | INFO     | silero_tts.silero_tts:init_model:187 - Loading model
2025-09-30 23:33:07.216 | INFO     | silero_tts.silero_tts:init_model:192 - Model to device takes 2.13 seconds
2025-09-30 23:33:07.218 | INFO     | silero_tts.silero_tts:init_model:195 - Synchronizing CUDA
2025-09-30 23:33:07.220 | INFO     | silero_tts.silero_tts:init_model:198 - Cuda Synch takes 0.00 seconds
2025-09-30 23:33:07.221 | SUCCESS  | silero_tts.silero_tts:init_model:199 - Model is loaded


['aidar', 'baya', 'kseniya', 'xenia', 'eugene', 'random']

Сделаем удобную функцию для синтеза, которая будет возвращать аудиосегменты, которые можно будет склеивать вместе:

In [29]:
import io
from pydub import AudioSegment

def synth(x,speaker=None):
  if speaker:
    tts.change_speaker(speaker)
  bio = io.BytesIO()
  tts.tts(x,bio)
  bio.seek(0)
  return AudioSegment(bio.getvalue())

synth('Привет')

2025-09-30 23:41:05.659 | INFO     | silero_tts.silero_tts:preprocess_text:234 - Preprocessing text
2025-09-30 23:41:05.663 | INFO     | silero_tts.silero_tts:init_wave_file:289 - Initializing wave file: <_io.BytesIO object at 0x7cef31a42d40>
2025-09-30 23:41:05.666 | INFO     | silero_tts.silero_tts:tts:271 - Starting TTS
2025-09-30 23:41:05.667 | INFO     | silero_tts.silero_tts:tts:274 - Processing line 1/1: Привет
2025-09-30 23:41:05.713 | SUCCESS  | silero_tts.silero_tts:tts:286 - Speech saved to <_io.BytesIO object at 0x7cef31a42d40>


Теперь можем озвучить диалог:

In [30]:
res = synth('Вот какой диалог получился однажды у учителя и ученика!','baya')
for x in teacher.messages[1:]:
  spk = 'xenia' if x['role']=='assistant' else 'aidar'
  res += synth(x['content'],spk)

res

2025-09-30 23:42:54.507 | SUCCESS  | silero_tts.silero_tts:change_speaker:133 - Speaker changed to: baya
2025-09-30 23:42:54.518 | INFO     | silero_tts.silero_tts:preprocess_text:234 - Preprocessing text
2025-09-30 23:42:54.527 | INFO     | silero_tts.silero_tts:init_wave_file:289 - Initializing wave file: <_io.BytesIO object at 0x7cef6ddb98a0>
2025-09-30 23:42:54.531 | INFO     | silero_tts.silero_tts:tts:271 - Starting TTS
2025-09-30 23:42:54.539 | INFO     | silero_tts.silero_tts:tts:274 - Processing line 1/1: Вот какой диалог получился однажды у учителя и ученика!
2025-09-30 23:42:59.339 | SUCCESS  | silero_tts.silero_tts:tts:286 - Speech saved to <_io.BytesIO object at 0x7cef6ddb98a0>
2025-09-30 23:42:59.343 | SUCCESS  | silero_tts.silero_tts:change_speaker:133 - Speaker changed to: aidar
2025-09-30 23:42:59.349 | INFO     | silero_tts.silero_tts:preprocess_text:234 - Preprocessing text
2025-09-30 23:42:59.350 | INFO     | silero_tts.silero_tts:init_wave_file:289 - Initializing w

## Мораль

Развёртывание своих моделей требует существенных усилий, и результат не всегда сразу хорош. Облако в этом смысле сильно упрощает жизнь, предоставляя готовые модели. Но развёртывая свои модели мы можем экспериментировать, важно лишь помнить об ограниченных вычислительных мощностях.